# Ejemplo usando FAISS

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025 

# Parte 1: Faiss CPU

Usaremos una librería publicada por Facebook Research llamada Faiss  https://github.com/facebookresearch/faiss/

Es una librería para C++ que tiene un wrapper para python.  
Tiene una versión que usa solo cpu (`faiss-cpu`) y una version que usa GPU (`faiss-gpu`).  

Los pasos para instalar se pueden revisar acá: https://github.com/facebookresearch/faiss/blob/main/INSTALL.md

```
conda install -c pytorch faiss-cpu=1.11.0
```

Requiere instalar python `3.11`, por lo que puede ser necesario crear un nuevo ambiente:

```
conda create -n cc5213_v2 python=3.11
conda activate cc5213_v2
conda install -c pytorch faiss-cpu=1.11.0
# para comparar
conda install jupyter pyflann 
```


### Leer los datos

In [ ]:
import numpy
import time


class ResultadoBusqueda:
    def __init__(self, nombre, distancias, nns, tiempo_indice, tiempo_busqueda):
        self.nombre = nombre
        self.distancias = distancias
        self.nns = nns
        self.tiempo_indice = tiempo_indice
        self.tiempo_busqueda = tiempo_busqueda


class Dataset:
    def __init__(self, nombre, archivo_q, archivo_r):
        self.nombre = nombre
        self.q = numpy.load(archivo_q)
        self.r = numpy.load(archivo_r)
        print("Dataset {}: Q={} R={}".format(self.nombre, self.q.shape, self.r.shape))


datasetA = Dataset("DATASET_A", "dataset_a_q.npy", "dataset_a_r.npy")

## Búsqueda 1-NN usando linear scan de FLANN

In [ ]:
import pyflann


def linear_scan_pyflann(dataset):
    # 1) crear indice
    t0 = time.time()
    flann = pyflann.FLANN()
    flann.build_index(dataset.r, algorithm="linear")
    t1 = time.time()
    # 2) busqueda del 1-NN
    nns, dists = flann.nn_index(dataset.q, num_neighbors=1, cores=1)
    t2 = time.time()
    return ResultadoBusqueda("PyFlann", dists, nns, t1 - t0, t2 - t1)


# demora unos 25 a 35 segundos
t1 = time.time()
LS_flann = linear_scan_pyflann(datasetA)
t2 = time.time()
print("tiempo={:.1f} seg.".format(t2 - t1))

## Búsqueda 1-NN usando linear scan de Faiss

In [ ]:
import faiss

# para efectos de comparación de tiempos, se limita a usar un solo hilo
# https://github.com/facebookresearch/faiss/wiki/Threads-and-asynchronous-calls
faiss.omp_set_num_threads(1)


def linear_scan_faiss_cpu(dataset):
    # 1) crear indice
    t0 = time.time()
    index = faiss.IndexFlatL2(dataset.r.shape[1])
    index.add(dataset.r)
    t1 = time.time()
    # 2) busqueda del 1-NN
    dists, nns = index.search(dataset.q, 1)
    t2 = time.time()
    return ResultadoBusqueda("Faiss-Cpu", dists, nns, t1 - t0, t2 - t1)


t1 = time.time()
LS_faiss_cpu = linear_scan_faiss_cpu(datasetA)
t2 = time.time()
print("tiempo={:.1f} seg.".format(t2 - t1))

#### Nota:
Este linear scan, en mi computador demora menos de 1 segundo (!?).

Es extraño, ya que el tiempo no cambia si se ajusta `omp_set_num_threads()`.  

**Posible razón:** Al revisar el [código fuente de distances.cpp](https://github.com/facebookresearch/faiss/blob/main/faiss/utils/distances.cpp) se ve que usa la librería BLAS para distancia euclidiana, podría ser que esa librería está usando alguna optimización para el cálculo de distancias que las otras librerías no tienen.


## Verificar que Faiss y PyFlann entregan los mismos NN

Notar que Faiss entrega un array de arrays, mientras que PyFlann entrega un array.   
Con `reshape()` se convierten ambos a un array que debe ser del mismo largo.

In [ ]:
def comparar(resultado_1, resultado_base):
    array1 = resultado_1.nns.reshape(len(resultado_base.nns))
    array2 = resultado_base.nns.reshape(len(resultado_base.nns))
    assert len(array1) == len(array2)
    total = len(array1)
    iguales = 0
    for i in range(total):
        if array1[i] == array2[i]:
            iguales += 1
    print(
        "{} vs {}:  iguales={} ({:.1%})  tiempo_indexar=({:.2f} vs {:.2f} segs.)  tiempo_busqueda=({:.2f} vs {:.2f} segs.)".format(
            resultado_1.nombre,
            resultado_base.nombre,
            iguales,
            iguales / total,
            resultado_1.tiempo_indice,
            resultado_base.tiempo_indice,
            resultado_1.tiempo_busqueda,
            resultado_base.tiempo_busqueda,
        )
    )


comparar(LS_faiss_cpu, LS_flann)

## Probando índice IndexIVFFlat

Según la documentación en https://github.com/facebookresearch/faiss/wiki/Faiss-indexes el índice IndexIVFFlat significa: "Inverted file with exact post-verification".

In [ ]:
def construir_indice_IVF_cpu(dataset, nlists):
    quantizer = faiss.IndexFlatL2(dataset.r.shape[1])
    index = faiss.IndexIVFFlat(quantizer, dataset.r.shape[1], nlists)
    index.train(dataset.r)
    index.add(dataset.r)
    return index


def buscar_IVF_cpu(dataset, nlists):
    # 1) crear indice
    t0 = time.time()
    index = construir_indice_IVF_cpu(dataset, nlists)
    t1 = time.time()
    # 2) busqueda del 1-NN
    dists, nns = index.search(dataset.q, 1)
    t2 = time.time()
    return ResultadoBusqueda(
        "IVF_cpu nlists-{:<3}".format(nlists), dists, nns, t1 - t0, t2 - t1
    )


for nlists in 2, 5, 10, 20, 50, 100:
    resultado = buscar_IVF_cpu(datasetA, nlists)
    comparar(resultado, LS_flann)

### Otros índices
Se puede probar con otros índices y combinaciones de ellos usando la función `index_factory`. Por ejemplo:
```
index = faiss.index_factory(dataset.r.shape[1], "IVF100,PQ8")
```
Ver ejemplos algunos índices en https://github.com/facebookresearch/faiss/wiki/Lower-memory-footprint



# Parte 2: Usando GPU

Usaremos la versión GPU de Faiss:
```
  conda install -c pytorch -c nvidia faiss-gpu=1.11.0
```

No es posible tener instalado `faiss-cpu`y `faiss-gpu`simultáneamente.  
Se puede crear un nuevo ambiente o desinstalar la versión cpu:
```
conda uninstall faiss-cpu
```

Para usar la GPU, el código fuente es el mismo que con cpu con la única diferencia es que hay que llamar a `faiss.index_cpu_to_all_gpus()` al construir un índice:
```
    index = ....
    gpu_index = faiss.index_cpu_to_all_gpus(index)
    gpu_index.add(....)
    return gpu_index
```
Ver https://github.com/facebookresearch/faiss/wiki/Running-on-GPUs

## Probar Linear Scan con GPU

Si aparece el siguiente error:
```
NameError: name 'GpuResourcesVector' is not defined
```
es porque el ambiente tiene instalado `faiss-cpu` y no `faiss-gpu`. Crear el ambiente nuevo y abrir este notebook en un ambiente con `faiss-gpu`.

In [ ]:
import faiss

ngpus = faiss.get_num_gpus()
print("cantidad de GPUs: {}".format(ngpus))


def linear_scan_faiss_gpu(dataset):
    # 1) crear indice
    t0 = time.time()
    index = faiss.IndexFlatL2(dataset.r.shape[1])
    # convertir a un indice con gpu
    gpu_index = faiss.index_cpu_to_all_gpus(index)
    gpu_index.add(dataset.r)
    t1 = time.time()
    # 2) busqueda del 1-NN
    dists, nns = gpu_index.search(dataset.q, 1)
    t2 = time.time()
    return ResultadoBusqueda("Faiss-GPU", dists, nns, t1 - t0, t2 - t1)


LS_faiss_gpu = linear_scan_faiss_gpu(datasetA)
comparar(LS_faiss_gpu, LS_flann)

Notar que la búsqueda del NN usando GPU es mucho más rápida que con CPU (en mi computador demora 0.2 segundos).  
Es mucho más rápida porque la GPU hace cientos o miles de cálculos en paralelo (depende de los cores de la GPU), mientras que la CPU puede usar unos pocos hilos (y la limitamos a 1).

## Probando índice IndexIVFFlat con GPU

Según la documentación en https://github.com/facebookresearch/faiss/wiki/Faiss-indexes el índice IndexIVFFlat significa: "Inverted file with exact post-verification".

In [ ]:
def construir_indice_IVF_gpu(dataset, nlists):
    quantizer = faiss.IndexFlatL2(dataset.r.shape[1])
    index = faiss.IndexIVFFlat(quantizer, dataset.r.shape[1], nlists)
    gpu_index = faiss.index_cpu_to_all_gpus(index)
    gpu_index.train(dataset.r)
    gpu_index.add(dataset.r)
    return gpu_index


def buscar_IVF_gpu(dataset, nlists):
    # 1) crear indice
    t0 = time.time()
    index = construir_indice_IVF_gpu(dataset, nlists)
    t1 = time.time()
    # 2) busqueda del 1-NN
    dists, nns = index.search(dataset.q, 1)
    t2 = time.time()
    return ResultadoBusqueda(
        "IVF_GPU nlists-{:<3}".format(nlists), dists, nns, t1 - t0, t2 - t1
    )


for nlists in 2, 5, 10, 20, 50, 100:
    resultado = buscar_IVF_gpu(datasetA, nlists)
    comparar(resultado, LS_flann)

Al usar GPU el tiempo del linear scan baja mucho, por lo que es difícil probar otros índices más sofisticados como IVF porque no se alcanza a notar la ganancia en tiempo.
